In [ ]:
import ollama
import json


class StrideIntentAnalyser:
    def __init__(
        self,
        model: str = "llama3.2:3b",
        temperature: float = 0.0,
    ):
        self.model = model
        self.temperature = temperature
        self.system_prompt = """
    You are a shoe support intent classifier. Analyze the user text and return ONLY a JSON object.
    You have to check Does this text describe damage caused by a pet, an accident, or user misuse?.
    and then classify them into one of these Categories:
    - refund_request: Money back.
    - return_request: Sending product back (fit/style).
    - replacement_request: Wanting a new pair of the same shoe.
    - repair_request: Fix for manufacturing defects.
    - manual_inspection: Damage due to manhandle, misuse, or accidents.
    - general_chat: Greetings or small  or anything else apart from context of shoe.

    Output Schema:
    {
        "intent": "string",
        "confidence": "integer (0-100)",
        "reason": "short explanation of why this category was chosen",
        "misuse_or_accident": "boolean"
    }
    """

    def generate_response(self, user_query: str):
        response = ollama.generate(
            model=self.model,
            system=self.system_prompt,
            prompt=f"User text: '{user_query}'",
            format="json",
            options={"temperature": self.temperature},
        )
        return json.loads(response["response"])

    def final_intent_system(self, user_text: str):
        # Step 1: Get desired intent
        intent_data = self.generate_response(user_query=user_text)
        final_intent = intent_data.get("intent", "general_chat")
        misuse_data = bool(intent_data.get("misuse_or_accident", False))
        confidence = int(intent_data.get("confidence", 0))
        reason = intent_data.get("reason", "No reason provided")

        if final_intent in {"refund_request", "return_request"}:
            return {
                "intent": "return_refund_request",
                "confidence": confidence,
                "misuse_or_accident": misuse_data,
                "reason": reason,
            }
        
        if final_intent in {"repair_request", "replacement_request"}:
            return {
                "intent": "replacement_repair_request",
                "confidence": confidence,
                "misuse_or_accident": misuse_data,
                "reason": reason,
            }

        # Correct override logic
        if final_intent not in {"manual_inspection", "general_chat"} and misuse_data:
            return {
                "intent": "manual_inspection",
                "confidence": confidence,
                "misuse_or_accident": misuse_data,
                "reason": f"Override: User requested {final_intent}, but damage was flagged as misuse",
            }

        return {
            "intent": final_intent,
            "confidence": confidence,
            "misuse_or_accident": misuse_data,
            "reason": reason,
        }

In [2]:
analyser = StrideIntentAnalyser()

cases = [
    "The sole came off after my dog chewed it",
    "I want a refund, the shoe was damaged in rain",
    "Hi there!",
    "The stitching came loose within a week",
    "The shoe doesn't fit, want to return it",
    "The phone is not working i want to get it repair",

]

for text in cases:
    print("\nINPUT:", text)
    print(analyser.final_intent_system(text))



INPUT: The sole came off after my dog chewed it
{'intent': 'manual_inspection', 'confidence': 90, 'misuse_or_accident': True, 'reason': 'Damage caused by pet chewing.'}

INPUT: I want a refund, the shoe was damaged in rain
{'intent': 'return_refund_request', 'confidence': 100, 'misuse_or_accident': False, 'reason': 'The user is requesting a refund due to damage caused by an external factor (rain)'}

INPUT: Hi there!
{'intent': 'general_chat', 'confidence': 100, 'misuse_or_accident': False, 'reason': 'Greetings or small talk, not related to shoe damage or request.'}

INPUT: The stitching came loose within a week
{'intent': 'manual_inspection', 'confidence': 90, 'misuse_or_accident': True, 'reason': 'Damage caused by user misuse or accident, as the product was used for a short period and the stitching came loose.'}

INPUT: The shoe doesn't fit, want to return it
{'intent': 'return_refund_request', 'confidence': 100, 'misuse_or_accident': False, 'reason': 'The user is requesting a return